In [32]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression


df = pd.read_csv(r'C:\Users\admin\Desktop\Macbook\TD\test_TDmodel.csv',skip_blank_lines=True, encoding='cp1252')


# Preprocess timestamp column (PERIOD)
df['period'] = pd.to_datetime(df['period'])  # Convert to datetime data type
df['Year'] = df['period'].dt.year  # Extract year as a feature
df['Month'] = df['period'].dt.month  # Extract month as a feature

df

,period,PRODUCT_GROUP,movement_type,Term_CM,Term_LM,Currency_2,TD_type,NET,EOP_CM,EOP_LM,CIE_VOL,ADB_VOL,VOF_VOL,NII_VOL,Year,Month
0,2019-10-01,TD,3ROLL,From 18M,From 18M,NON-VND,NOT_ROLL,0.004584,13.991939,13.987355,-0.035624,1.279339e+04,39.051442,0,2019,10
1,2023-09-01,TD,6ROLL_AUTOROLL,From 18M,From 18M,VND,AUTOROLL,0.426353,4.435844,4.009491,-1008.009941,1.406047e+05,1262.098360,0,2023,9
2,2023-04-01,TD,4UNCHANGED_AUTOROLL,1-3M,1-3M,VND,AUTOROLL,0.000000,1979.954323,1979.954323,-5450.417331,1.231275e+06,6299.548081,0,2023,4
3,2022-08-01,TD,4SCHEDULE,less than 1M,less than 1M,NON-VND,NOT_ROLL,-26.287476,0.000000,26.287476,-0.013163,2.379709e+04,43.897450,0,2022,8
4,2023-05-01,TD,1NEW_NOTROLL,1-3M,NaN,VND,NOT_ROLL,1984.856350,1984.856350,0.000000,-5348.084365,1.243732e+06,6299.728834,0,2023,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4014,2022-11-01,TD,2NEW,12-18M,NaN,NON-VND,NOT_ROLL,4.372662,4.372662,0.000000,-2.431596,2.960490e+04,34.347614,0,2022,11
4015,2023-05-01,TD,5NEW_AUTOROLL,1-3M,NaN,NON-VND,AUTOROLL,190.149978,190.149978,0.000000,-0.853443,3.141269e+04,76.896378,0,2023,5
4016,2023-12-01,TD,1NEW_NOTROLL,10-11M,NaN,VND,NOT_ROLL,6.761161,6.761161,0.000000,0.000000,0.000000e+00,0.000000,0,2023,12
4017,2022-12-01,TD,3ROLL,less than 1M,less than 1M,NON-VND,NOT_ROLL,-0.225411,5.564832,5.790243,-0.027223,2.378079e+04,47.787189,0,2022,12


## Tập dữ liệu 1

Mô hình linear regression dự đoán NET dựa trên tập dữ liệu gồm:
- Period
- Movement_type
- Term_LM
- Term_CM
- EOP_LM
- EOP_CM

Cho ra kết quả tốt nhất tại: 

In [26]:
import pandas as pd
from sklearn.metrics import mean_squared_error

# Chuẩn bị các đặc trưng đầu vào (X) và biến mục tiêu (y)
X = encoded_df.drop(['NET', 'period','CIE_VOL','ADB_VOL', 'VOF_VOL', 'NII_VOL'], axis=1)
y = encoded_df['NET']

best_r2 = -1
best_seed = None

results = []

for size in (0.6, 0.7, 0.8, 0.9):
    best_r2_train = -1
    best_r2_test = -1
    best_seed = None
    
    for seed in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, random_state=seed)
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        
        if r2_train > best_r2_train:
            best_r2_train = r2_train
            best_r2_test = r2_test
            best_seed = seed
    
    results.append([size, best_r2_train, best_r2_test, mse_test, best_seed])


df_results = pd.DataFrame(results, columns=['Train Size', 'Best R-squared Train', 'Best R-squared Test', 'Mean Square Error', 'Best Seed'])
df_results

NameError: name 'encoded_df' is not defined

## Tập dữ liệu 2

Mô hình linear regression dự đoán NET dựa trên tập dữ liệu gồm:
- Period
- Movement_type
- Term_LM
- Term_CM
- EOP_LM

In [35]:
df_2 = df.drop(['EOP_CM', 'CIE_VOL', 'ADB_VOL', 'VOF_VOL', 'NII_VOL'], axis=1)
df_2 = df_2.groupby(['period', 'PRODUCT_GROUP', 'movement_type', 'NET', 'Term_CM', 'Term_LM', 'Currency_2', 'TD_type', 'Year', 'Month']).agg({'EOP_LM': 'sum'}).reset_index()
df_2 = df_2[df_2['NET'] != 0]

categorical_cols = ['PRODUCT_GROUP', 'movement_type', 'Term_CM', 'Term_LM', 'Currency_2', 'TD_type']

# Encode categorical variables using one-hot encoding
encoded_df = pd.get_dummies(df_2, columns=categorical_cols, drop_first=True)   
encoded_df

,period,NET,Year,Month,EOP_LM,movement_type_2SCHE_NOTROLL,movement_type_3PREMATURE_NOTROLL,movement_type_3ROLL,movement_type_4PREMATURE,movement_type_4SCHEDULE,...,Term_CM_From 18M,Term_CM_less than 1M,Term_LM_10-11M,Term_LM_12-18M,Term_LM_4-5M,Term_LM_6-9M,Term_LM_From 18M,Term_LM_less than 1M,Currency_2_VND,TD_type_NOT_ROLL
8,2018-11-01,-0.003207,2018,11,2.995163,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True
9,2018-11-01,-0.000324,2018,11,0.302948,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,True
10,2018-11-01,0.000443,2018,11,0.577493,False,False,False,False,False,...,False,True,False,False,False,False,False,True,True,True
11,2018-11-01,0.026825,2018,11,0.826300,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,True
12,2018-11-01,0.036228,2018,11,2.579403,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2647,2024-01-01,23.131174,2024,1,34.797482,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
2648,2024-01-01,75.055067,2024,1,59.752452,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,True
2649,2024-01-01,90.783778,2024,1,603.333900,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False
2650,2024-01-01,174.828353,2024,1,175.181294,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False


In [36]:
# Chuẩn bị các đặc trưng đầu vào (X) và biến mục tiêu (y)
X = encoded_df.drop(['NET', 'period','EOP_LM'], axis=1)
y = encoded_df['NET']

best_r2 = -1
best_seed = None

results = []

for size in (0.6, 0.7, 0.8, 0.9):
    best_r2_train = -1
    best_r2_test = -1
    best_seed = None
    
    for seed in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, random_state=seed)
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        
        if r2_train > best_r2_train:
            best_r2_train = r2_train
            best_r2_test = r2_test
            best_seed = seed
    
    results.append([size, best_r2_train, best_r2_test, mse_test, best_seed])


df_results = pd.DataFrame(results, columns=['Train Size', 'Best R-squared Train', 'Best R-squared Test', 'Mean Square Error', 'Best Seed'])
df_results

,Train Size,Best R-squared Train,Best R-squared Test,Mean Square Error,Best Seed
0,0.6,0.352940,0.151322,1.418036e+07,95
1,0.7,0.355385,0.105840,1.247075e+07,31
2,0.8,0.336446,0.107518,7.321289e+06,21
3,0.9,0.300947,0.098468,7.989329e+06,67


## Tập dữ liệu 3

Mô hình linear regression dự đoán EOP_CM dựa trên tập dữ liệu gồm:
- Period
- Movement_type
- Term_LM
- Term_CM
- EOP_LM

In [37]:
df_2 = df.drop(['NET', 'CIE_VOL', 'ADB_VOL', 'VOF_VOL', 'NII_VOL'], axis=1)

categorical_cols = ['PRODUCT_GROUP', 'movement_type', 'Term_CM', 'Term_LM', 'Currency_2', 'TD_type']

# Encode categorical variables using one-hot encoding
encoded_df = pd.get_dummies(df_2, columns=categorical_cols, drop_first=True)   
encoded_df

,period,EOP_CM,EOP_LM,Year,Month,movement_type_1NEW_NOTROLL,movement_type_1UNCHANGED,movement_type_2NEW,movement_type_2SCHE_NOTROLL,movement_type_3PREMATURE_NOTROLL,...,Term_CM_From 18M,Term_CM_less than 1M,Term_LM_10-11M,Term_LM_12-18M,Term_LM_4-5M,Term_LM_6-9M,Term_LM_From 18M,Term_LM_less than 1M,Currency_2_VND,TD_type_NOT_ROLL
0,2019-10-01,13.991939,13.987355,2019,10,False,False,False,False,False,...,True,False,False,False,False,False,True,False,False,True
1,2023-09-01,4.435844,4.009491,2023,9,False,False,False,False,False,...,True,False,False,False,False,False,True,False,True,False
2,2023-04-01,1979.954323,1979.954323,2023,4,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,2022-08-01,0.000000,26.287476,2022,8,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,True
4,2023-05-01,1984.856350,0.000000,2023,5,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4014,2022-11-01,4.372662,0.000000,2022,11,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
4015,2023-05-01,190.149978,0.000000,2023,5,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4016,2023-12-01,6.761161,0.000000,2023,12,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
4017,2022-12-01,5.564832,5.790243,2022,12,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,True


In [38]:
# Chuẩn bị các đặc trưng đầu vào (X) và biến mục tiêu (y)
X = encoded_df.drop(['period','EOP_CM'], axis=1)
y = encoded_df['EOP_CM']

best_r2 = -1
best_seed = None

results = []

for size in (0.6, 0.7, 0.8, 0.9):
    best_r2_train = -1
    best_r2_test = -1
    best_seed = None
    
    for seed in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, random_state=seed)
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        
        if r2_train > best_r2_train:
            best_r2_train = r2_train
            best_r2_test = r2_test
            best_seed = seed
    
    results.append([size, best_r2_train, best_r2_test, mse_test, best_seed])


df_results = pd.DataFrame(results, columns=['Train Size', 'Best R-squared Train', 'Best R-squared Test', 'Mean Square Error', 'Best Seed'])
df_results

,Train Size,Best R-squared Train,Best R-squared Test,Mean Square Error,Best Seed
0,0.6,0.932889,0.844874,7.052975e+06,30
1,0.7,0.930522,0.806187,8.082917e+06,30
2,0.8,0.928262,0.737813,8.788571e+06,27
3,0.9,0.927956,0.549735,1.388977e+07,27


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import pandas as pd

best_r2 = -1
best_seed = None

results = []

for size in (0.6, 0.7, 0.8, 0.9):
    best_r2_train = -1
    best_r2_test = -1
    best_seed = None
    
    for seed in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, random_state=seed)
        
        # Tạo và huấn luyện mô hình XGBoost
        model = xgb.XGBRegressor()
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        
        if r2_train > best_r2_train:
            best_r2_train = r2_train
            best_r2_test = r2_test
            best_seed = seed
    
    results.append([size, best_r2_train, best_r2_test, mse_test, best_seed])

df_results = pd.DataFrame(results, columns=['Train Size', 'Best R-squared Train', 'Best R-squared Test', 'Mean Square Error', 'Best Seed'])
df_results

,Train Size,Best R-squared Train,Best R-squared Test,Mean Square Error,Best Seed
0,0.6,0.994447,0.908735,5.151016e+06,5
1,0.7,0.993750,0.880267,5.088288e+06,5
2,0.8,0.992430,0.775804,7.101064e+06,13
3,0.9,0.992042,0.856840,1.101847e+07,13


## Tập dữ liệu 4
Mô hình linear regression dự đoán EOP_CM dựa trên tập dữ liệu gồm:
- Period
- Term_CM
- EOP_LM

In [43]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression


df_v2 = pd.read_csv(r'C:\Users\admin\Desktop\Macbook\TD\test_TDmodel_V2.csv',skip_blank_lines=True, encoding='cp1252')
df_IR = pd.read_csv(r'C:\Users\admin\Desktop\Macbook\TD\test_TDmodel_IR.csv',skip_blank_lines=True, encoding='cp1252')

# Preprocess timestamp column (PERIOD)
df_v2['period'] = pd.to_datetime(df_v2['period'])  # Convert to datetime data type
df_v2['Year'] = df_v2['period'].dt.year  # Extract year as a feature
df_v2['Month'] = df_v2['period'].dt.month  # Extract month as a feature

df_v2

,period,Term_CM,Currency_2,NET,EOP_CM,EOP_LM,CIE_VOL,ADB_VOL,VOF_VOL,NII_VOL,Year,Month
0,2019-10-01,From 18M,NON-VND,2.418886e+01,166.870643,142.681786,-0.106871,3.838018e+04,117.154325,0,2019,10
1,2023-09-01,From 18M,VND,4.263527e-01,4.435844,4.009491,-1008.009941,1.406047e+05,1262.098360,0,2023,9
2,2023-04-01,1-3M,VND,7.178225e+03,16171.603180,8993.378469,-81756.259970,1.846913e+07,94493.221220,0,2023,4
3,2022-08-01,less than 1M,NON-VND,-2.628748e+01,0.000000,26.287476,-0.013163,2.379709e+04,43.897450,0,2022,8
4,2023-05-01,1-3M,VND,6.716870e+03,15992.202470,9275.332205,-69525.096750,1.616852e+07,81896.474840,0,2023,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2740,2019-02-01,less than 1M,VND,8.310000e-07,0.010770,0.010769,-1837.154230,1.002363e+06,1916.531910,0,2019,2
2741,2020-06-01,6-9M,VND,9.444006e+03,9444.005663,0.000000,-37420.933590,6.301759e+06,40836.894690,0,2020,6
2742,2020-09-01,NaN,VND,2.708817e-01,3.203403,2.932522,0.000000,0.000000e+00,0.000000,0,2020,9
2743,2023-05-01,1-3M,NON-VND,1.901500e+02,190.149978,0.000000,-0.853443,3.141269e+04,76.896378,0,2023,5


##### a. Tìm r^2 tốt nhất

In [49]:
df_2 = df_v2.drop(['period', 'NET','CIE_VOL', 'ADB_VOL', 'VOF_VOL', 'NII_VOL'], axis=1)
# df_v2 = df_v2[df_v2['Term_CM'] == 'less than 1M']

categorical_cols = [ 'Term_CM',  'Currency_2']

# Encode categorical variables using one-hot encoding
encoded_df = pd.get_dummies(df_v2, columns=categorical_cols, drop_first=True)   

X = encoded_df.drop(['EOP_CM','period', 'NET','CIE_VOL', 'ADB_VOL', 'VOF_VOL', 'NII_VOL'], axis=1)
y = encoded_df['EOP_CM']

best_r2 = -1
best_seed = None

results = []

for size in (0.6, 0.7, 0.8, 0.9):
    best_r2_train = -1
    best_r2_test = -1
    best_seed = None
    
    for seed in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, random_state=seed)
        
        # Tạo và huấn luyện mô hình XGBoost
        model = xgb.XGBRegressor()
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        
        if r2_train > best_r2_train:
            best_r2_train = r2_train
            best_r2_test = r2_test
            best_seed = seed
    
    results.append([size, best_r2_train, best_r2_test, mse_test, best_seed])

df_results = pd.DataFrame(results, columns=['Train Size', 'Best R-squared Train', 'Best R-squared Test', 'Mean Square Error', 'Best Seed'])
df_results

,Train Size,Best R-squared Train,Best R-squared Test,Mean Square Error,Best Seed
0,0.6,0.997440,0.933714,1.553214e+07,42
1,0.7,0.996140,0.942185,1.625649e+07,69
2,0.8,0.996005,0.921792,2.042342e+07,69
3,0.9,0.993228,0.890458,1.083729e+07,88


In [52]:
best_r2 = -1
best_seed = None

results = []

for size in (0.6, 0.7, 0.8, 0.9):
    best_r2_train = -1
    best_r2_test = -1
    best_seed = None
    
    for seed in range(100):
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=size, random_state=seed)
        
        # Tạo và huấn luyện mô hình XGBoost
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        mse_test = mean_squared_error(y_test, y_test_pred)
        
        if r2_train > best_r2_train:
            best_r2_train = r2_train
            best_r2_test = r2_test
            best_seed = seed
    
    results.append([size, best_r2_train, best_r2_test, mse_test, best_seed])

df_results = pd.DataFrame(results, columns=['Train Size', 'Best R-squared Train', 'Best R-squared Test', 'Mean Square Error', 'Best Seed'])
df_results

,Train Size,Best R-squared Train,Best R-squared Test,Mean Square Error,Best Seed
0,0.6,0.932857,0.880595,2.047706e+07,81
1,0.7,0.932416,0.847000,2.143533e+07,81
2,0.8,0.929766,0.817671,2.688566e+07,23
3,0.9,0.927917,0.740054,2.642824e+07,83


#### b. Tìm mse tốt nhất

In [23]:
df_ir = pd.read_csv(r'C:\Users\admin\Desktop\Macbook\TD\test_TDmodel_IR.csv',skip_blank_lines=True, encoding='cp1252')
df_ir['BUSINESS_DATE'] = pd.to_datetime(df_ir['BUSINESS_DATE'])
df_ir['Year'] = df_ir['BUSINESS_DATE'].dt.year  # Extract year as a feature
df_ir['Month'] = df_ir['BUSINESS_DATE'].dt.month  # Extract month as a feature

df_ir_max = df_ir.groupby(["Term_CM", "SEGMENT", "BUSINESS_DATE",'Year','Month'])["INT_VALUE"].max().reset_index()
df_ir_vpb = df_ir[df_ir['BANK_CODE'] == 'VPB'].groupby(["Term_CM", "SEGMENT", "BUSINESS_DATE",'Year','Month'])["INT_VALUE"].min().reset_index()
df_ir_vpb, df_ir_max



(          Term_CM SEGMENT BUSINESS_DATE  Year  Month  INT_VALUE
 0            1-3M      AF    2022-07-18  2022      7        4.0
 1            1-3M      AF    2022-08-01  2022      8        4.0
 2            1-3M      AF    2022-09-05  2022      9        4.0
 3            1-3M      AF    2022-10-03  2022     10        4.9
 4            1-3M      AF    2022-11-07  2022     11        6.0
 ..            ...     ...           ...   ...    ...        ...
 247  less than 1M   UMASS    2023-10-02  2023     10        5.8
 248  less than 1M   UMASS    2023-11-06  2023     11        5.3
 249  less than 1M   UMASS    2023-12-01  2023     12        5.3
 250  less than 1M   UMASS    2024-01-01  2024      1        5.0
 251  less than 1M   UMASS    2024-02-02  2024      2        5.0
 
 [252 rows x 6 columns],
           Term_CM SEGMENT BUSINESS_DATE  Year  Month  INT_VALUE
 0            1-3M      AF    2022-07-18  2022      7        4.0
 1            1-3M      AF    2022-08-01  2022      8        4.

In [24]:
# Lọc df_ir_max theo điều kiện segment = "UMASS" và tháng năm tương ứng
filtered_df_ir_max = df_ir_max[(df_ir_max["SEGMENT"] == "UMASS") & (df_ir_max["Month"].dt.month == df_v2["Month"]) & (df_ir_max["Year"].dt.year == df_v2["Year"])]

# Thực hiện left join giữa df_v2 và filtered_df_ir_max
df_combine = pd.merge(df_v2, filtered_df_ir_max[["term_cm", "INT_VALUE"]], left_on=["term_cm"], right_on=["term_cm"], how="left")


AttributeError: Can only use .dt accessor with datetimelike values